<a href="https://colab.research.google.com/github/seecode4/seeRepo1/blob/main/mec2-projects/Student_MLE_MiniProject_DL_HyperParamTuning_try2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q scikeras

import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import RocCurveDisplay

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.initializers import RandomNormal, RandomUniform
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 48.2 MB/s eta 0:00:00


In [2]:
DATA_PATH = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

# Download the dataset and load it into a pandas DataFrame
df_data = pd.read_csv(DATA_PATH, header=None)

In [3]:
# Display the first few rows of the DataFrame
print(df_data.head())
print(df_data.shape)

   0                  1       2           3   4                    5   \
0  39          State-gov   77516   Bachelors  13        Never-married   
1  50   Self-emp-not-inc   83311   Bachelors  13   Married-civ-spouse   
2  38            Private  215646     HS-grad   9             Divorced   
3  53            Private  234721        11th   7   Married-civ-spouse   
4  28            Private  338409   Bachelors  13   Married-civ-spouse   

                   6               7       8        9     10  11  12  \
0        Adm-clerical   Not-in-family   White     Male  2174   0  40   
1     Exec-managerial         Husband   White     Male     0   0  13   
2   Handlers-cleaners   Not-in-family   White     Male     0   0  40   
3   Handlers-cleaners         Husband   Black     Male     0   0  40   
4      Prof-specialty            Wife   Black   Female     0   0  40   

               13      14  
0   United-States   <=50K  
1   United-States   <=50K  
2   United-States   <=50K  
3   United-State

In [4]:
# Ref: https://archive.ics.uci.edu/dataset/2/adult
feat_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
             'marital-status', 'occupation', 'relationship', 'race', 'sex',
             'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
             'income']
# Combine both
# help(df_data.set_axis)
df_raw = df_data.set_axis(feat_names, axis='columns')
print(df_raw.head())

   age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  income  
0          2174             0              40   United-States   <=50

In [5]:
# Do some exploratory analysis. How many rows/columns are there? How are NULL
# values represented? What's the percentrage of positive cases in the dataset?
num_rows = df_raw.shape[0]
num_cols = df_raw.shape[1]
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")
print("Column, i.e, feature names:\n", df_raw.columns[0:-1])
print("Target:", df_raw.columns[-1], "\n")
print("Column Data Types:\n", df_raw.dtypes)
print("\nNumber of isnull:\n", df_raw.isnull().sum())
print("Number of isna:\n", df_raw.isna().sum())
print("\nValue Counts for each feature:")
for featname in df_raw.columns:
  print(f"\n-----{featname}-----\n {df_raw[featname].value_counts()}")
target_value_counts = list(df_raw['income'].value_counts())
income_le_50k_pcnt = 100 * target_value_counts[0]/num_rows
income_gt_50k_pcnt = 100 * target_value_counts[1]/num_rows
print(f"\nPercentage of people with income <= 50K: {income_le_50k_pcnt:0.2f}%")
print(f"Percentage of people with income > 50K: {income_gt_50k_pcnt:0.2f}%")

Number of rows: 32561
Number of columns: 15
Column, i.e, feature names:
 Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')
Target: income 

Column Data Types:
 age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

Number of isnull:
 age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-

In [6]:
# Find all NULL values and drop them
# Check if value_counts match for this check
df = df_raw
print("Unknown Occupation Before drop null:", (df.loc[:,"occupation"].str.strip() == '?').sum())
for featname in df.columns:
  # string columns are of type 'object'
  if (df[featname].dtype == 'object'):
    # drop all rows that contain '?' which is the same as NULL here
    df = df.drop(df[df[featname].str.strip() == '?'].index)
print("Unknown Occupation After drop null:", (df.loc[:,"occupation"].str.strip() == '?').sum())

print("df shape after drop null:", df.shape)

Unknown Occupation Before drop null: 1843
Unknown Occupation After drop null: 0
df shape after drop null: (30162, 15)


In [7]:
# Use Scikit-Learn's LabelEncoder to convert the income column with a data type
# string to a binary variable.
le = LabelEncoder()
df['income'] = le.fit_transform(df['income'])
print(df['income'].value_counts())

income
0    22654
1     7508
Name: count, dtype: int64


In [8]:
# Use Scikit-Learn's LabelEncoder to convert the income column with a data type
# string to a binary variable.
le_occ = LabelEncoder()
df_occ = pd.DataFrame(le.fit_transform(df['occupation']))
print(type(df_occ))
print(df_occ.value_counts())

<class 'pandas.core.frame.DataFrame'>
9     4038
2     4030
3     3992
0     3721
11    3584
7     3212
6     1966
13    1572
5     1350
4      989
12     912
10     644
8      143
1        9
Name: count, dtype: int64


In [9]:
# Split dataset into training and test sets
# Lst[ Initial : End : IndexJump ]
print(df.columns)
X = df.iloc[:, 0:-1]
y = df.loc[:, 'income']
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape)
print(X_test.shape)

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
(30162, 14)
(30162,)
(24129, 14)
(6033, 14)


In [10]:
import tensorflow as tf

note=""" In tensorflow, a random operation relies on two different seeds:
a global seed, set by tf.set_random_seed, and
an operation seed, provided as an argument to the operation.
"""
# Config to make results repeatable/deterministic
tf.config.experimental.enable_op_determinism()
np.random.seed(1)
SEED=43
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)

In [11]:
# Use Scikit-Learn's ColumnTransformer to apply One Hot Encoding to the
# categorical variables in workclass, education, marital-status, occupation,
# relationship, 'race', sex, and native-country. #Also, apply MinMaxScaler to
# the remaining continuous features.
note = """
Each transformer is a three-element tuple that defines the name of the transformer,
 the transform to apply, and the column indices to apply it to.
For example:
(Name, Object, Columns)
transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(), [0, 1])])
"""
dts = X.dtypes

def get_col_indexes(df, col_names):
  return [df.columns.get_loc(name) for name in col_names]

# Categorical features and corresponding column index
cat_columns = [k for k,v in dts.items() if (v == 'object')]
cat_ix = get_col_indexes(X, cat_columns)

# Continuous features and corresponding column index
cont_columns = [k for k,v in dts.items() if (v != 'object')]
cont_ix = get_col_indexes(X, cont_columns)
print("cat_ix:", cat_ix, " cont_ix:", cont_ix)

# data preparation for categorical and continuous features
t = [('cat', OneHotEncoder(handle_unknown='ignore'), cat_ix),
     ('cont', MinMaxScaler(), cont_ix)]
ct = ColumnTransformer(transformers=t)

cat_ix: [1, 3, 5, 6, 7, 8, 9, 13]  cont_ix: [0, 2, 4, 10, 11, 12]


In [12]:
%%time
# Function to create model, required for KerasClassifier
def create_model(input_dim, num_layers=2, num_neurons=8, act_function='relu'):
  """With input_dim given create keras Sequential model with 3 hidden layers of 8, 16 and 8 neurons"""
  # create model
  initializer = RandomNormal(mean=0.0, stddev=1.0, seed=SEED)
  # initializer = RandomUniform(minval=0.0, maxval=1.0, seed=SEED)
  model = Sequential()
  model.add(tf.keras.Input(shape=(input_dim,)))
  for i in range(num_layers):
      model.add(Dense(num_neurons, activation=act_function,
                      kernel_initializer=initializer,
                      use_bias=False))
      # Add a BatchNormalization layer
      # model.add(BatchNormalization())

  model.add(Dense(1, activation='sigmoid', kernel_initializer=initializer,))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11.9 µs


In [13]:
# Basic Model without pipeline

# Column Transformer fit transform on X
X_trans = ct.fit_transform(X)
ct_feat_names = ct.get_feature_names_out()
print(X_trans.shape, type(X_trans))
print("Number of columns after transformation:", ct_feat_names.shape[0])

# Simple keras "Sequential" model with 1 hidden layer
Xt_train, Xt_test, yt_train, yt_test = train_test_split(X_trans, y, test_size=0.25, random_state=1)
print(Xt_train.shape, yt_train.shape)

num_feat = Xt_train.shape[1]
print("Num features in input for model:", type(num_feat), num_feat)

# Define the Keras model
basic_model = create_model(num_feat, num_layers=1, num_neurons=8, act_function='relu')
basic_model.summary()

# Fit to data
act_result = basic_model.fit(Xt_train, yt_train, epochs=10, batch_size=64)

# Test if the model is well assembled by predicting before training
yt_pred = basic_model.predict(Xt_train)
print("y_pred shape", yt_pred.shape, "\n First 3 values", yt_pred[0:3])
make_bin = lambda val: 1 if val > 0.5 else 0
yt_pred_df = pd.DataFrame(np.array([make_bin(yi) for yi in yt_pred]))
print(type(yt_pred_df), yt_pred_df.shape, "\n",
      yt_pred_df.value_counts(), yt_train.value_counts())

# Test, Loss and accuracy
loss_and_metrics = basic_model.evaluate(Xt_test, yt_test)
print('Loss = ',loss_and_metrics[0])
print('Accuracy = ',loss_and_metrics[1])

yt_test_pred = basic_model.predict(Xt_test)
basic_score = roc_auc_score(yt_test, yt_test_pred)
print(f"\nroc_auc_score with basic_model: {basic_score:0.3f}")

(30162, 104) <class 'scipy.sparse._csr.csr_matrix'>
Number of columns after transformation: 104
(22621, 104) (22621,)
Num features in input for model: <class 'int'> 104


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7419 - loss: 0.6281
Epoch 2/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7707 - loss: 0.4849
Epoch 3/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7877 - loss: 0.4368
Epoch 4/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8094 - loss: 0.4029
Epoch 5/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8216 - loss: 0.3829
Epoch 6/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8271 - loss: 0.3717
Epoch 7/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8305 - loss: 0.3647
Epoch 8/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8337 - loss: 0.3600
Epoch 9/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8357 - loss: 0.3564
Epoch 10/10
354/354 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8372 - loss: 0.3536
707/707 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
y_pred shape (22621, 1) 
 First 3 values [[0.5420019 ]
 [0.01367371]
 [0.68324465]]
<class 'pandas

In [14]:
%%time
# define create_model(input_dim, num_layers=2, num_neurons=8, act_function='relu')

# Hpyer parameter Tuning
def create_pipeline(m_input_dim, m_nl, m_nn, m_af):
  # Create KerasClassifier
  kerasclf_model = KerasClassifier(model=create_model, verbose=0,
                                   input_dim=m_input_dim,
                                   num_layers=m_nl,
                                   num_neurons=m_nn,
                                   act_function=m_af)
  # Create the scikit-learn pipeline ct uses OneHotEncoder
  kpipe = Pipeline(steps=[
      ('prep', ct),
       ('clf', kerasclf_model)
       ])
  return kpipe

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11.2 µs


In [15]:
%%time
import time

# With given hyperparam list fit, predict and get roc_auc_score
def kpipe_fit_pred_score(n_ep, n_bsize, num_pipe_feat, m_nl, m_nn, m_af):
  """Expect X_train, X_test, y_train, y_test already there;
  create_pipeline, fit, predict and get roc_auc_score """
  print(f"Tune with epochs={n_ep}  batch_size={n_bsize}",
        f"nlayers={m_nl}  nunits={m_nn} actfunc={m_af}")
  kpipe = create_pipeline(num_pipe_feat, m_nl, m_nn, m_af)
  start_time = time.time()
  # fit data to pipeline model
  history = kpipe.fit(X_train, y_train,
                      clf__epochs=n_ep, clf__batch_size=n_bsize)
  # time to train
  kclf_exectime = (time.time() - start_time)
  # predict test data
  y_test_pipe_pred = kpipe.predict(X_test)
  # get roc_auc_score
  kclf_score = (roc_auc_score(y_test, y_test_pipe_pred))
  return kclf_score, kclf_exectime

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 13.4 µs


In [16]:
# Try differen values for
# [n_epochs, n_batchsize, num_pipe_feat, m_nlayers, m_nunits, m_actfunc]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape, y_train.shape)

hplist1 = [[10, 64, 1, 8, 'relu'], [40, 64, 3, 8, 'tanh'],
           [40, 64, 3, 8, 'sigmoid'],  [40, 64, 2, 8, 'relu'],
           [80, 64, 2, 16, 'relu'], [80, 64, 2, 64, 'relu'],
           [100, 64, 4, 8, 'sigmoid'], [250, 256, 3, 8, 'sigmoid'],
           [250, 512, 4, 8, 'relu'], [500, 64, 4, 8, 'relu'],
           [500, 256, 3, 8, 'sigmoid']]
kclf_scores = []
kclf_exectimes = []
num_pipe_feat = 104
numsets1 = len(hplist1)
print("Number of hyperparam sets:", numsets1)
for i in range(numsets1):
  hp = hplist1[i]
  kclf_score, kclf_exectime = kpipe_fit_pred_score(hp[0], hp[1],
                                                   num_pipe_feat,
                                                   hp[2], hp[3], hp[4])
  kclf_scores.append(kclf_score)
  kclf_exectimes.append(kclf_exectime)
  print(f"Model Index: {i}, epochs={hp[0]}  batch_size={hp[1]}",
        f"nlayers={hp[2]}  nunits={hp[3]}  actfunc={hp[4]}",
        f"score={kclf_scores[i]:0.4f} exectime={kclf_exectimes[i]:0.4f}")

(24129, 14) (24129,)
Number of hyperparam sets: 11
Tune with epochs=10  batch_size=64 nlayers=1  nunits=8 actfunc=relu
Model Index: 0, epochs=10  batch_size=64 nlayers=1  nunits=8  actfunc=relu score=0.7638 exectime=9.4029
Tune with epochs=40  batch_size=64 nlayers=3  nunits=8 actfunc=tanh
Model Index: 1, epochs=40  batch_size=64 nlayers=3  nunits=8  actfunc=tanh score=0.7863 exectime=33.8722
Tune with epochs=40  batch_size=64 nlayers=3  nunits=8 actfunc=sigmoid
Model Index: 2, epochs=40  batch_size=64 nlayers=3  nunits=8  actfunc=sigmoid score=0.7926 exectime=31.7120
Tune with epochs=40  batch_size=64 nlayers=2  nunits=8 actfunc=relu
Model Index: 3, epochs=40  batch_size=64 nlayers=2  nunits=8  actfunc=relu score=0.7769 exectime=30.5688
Tune with epochs=80  batch_size=64 nlayers=2  nunits=16 actfunc=relu
Model Index: 4, epochs=80  batch_size=64 nlayers=2  nunits=16  actfunc=relu score=0.7911 exectime=58.0039
Tune with epochs=80  batch_size=64 nlayers=2  nunits=64 actfunc=relu
Model In

In [17]:
# Scores with different tries
print("Scores with Various HyperParameter Settings")
numsets = numsets1
hplist = hplist1
for i in range(numsets):
  if (kclf_scores[i] > 0.7):
      print(f"Model Index: {i}, epochs={hplist[i][0]}  batch_size={hplist[i][1]}",
            f"  nlayers={hplist[i][2]}  nunits={hplist[i][3]}  actfunc={hplist[i][4]} ",
            f"  score={kclf_scores[i]:0.4f}  exectime={kclf_exectimes[i]:0.4f}")

Scores with Various HyperParameter Settings
Model Index: 0, epochs=10  batch_size=64   nlayers=1  nunits=8  actfunc=relu    score=0.7638  exectime=9.4029
Model Index: 1, epochs=40  batch_size=64   nlayers=3  nunits=8  actfunc=tanh    score=0.7863  exectime=33.8722
Model Index: 2, epochs=40  batch_size=64   nlayers=3  nunits=8  actfunc=sigmoid    score=0.7926  exectime=31.7120
Model Index: 3, epochs=40  batch_size=64   nlayers=2  nunits=8  actfunc=relu    score=0.7769  exectime=30.5688
Model Index: 4, epochs=80  batch_size=64   nlayers=2  nunits=16  actfunc=relu    score=0.7911  exectime=58.0039
Model Index: 5, epochs=80  batch_size=64   nlayers=2  nunits=64  actfunc=relu    score=0.7256  exectime=81.6418
Model Index: 6, epochs=100  batch_size=64   nlayers=4  nunits=8  actfunc=sigmoid    score=0.8000  exectime=91.2745
Model Index: 7, epochs=250  batch_size=256   nlayers=3  nunits=8  actfunc=sigmoid    score=0.7753  exectime=76.7963
Model Index: 8, epochs=250  batch_size=512   nlayers=4 

In [18]:
# Linear Regression with keras

# Function to create model, required for KerasClassifier
def create_model_LinRegr(input_dim):
  """With input_dim given create keras Sequential model with 3 hidden layers of 8, 16 and 8 neurons"""
  # create model
  # initializer = RandomNormal(mean=0.0, stddev=1.0, seed=SEED)
  initializer = RandomUniform(minval=0.0, maxval=1.0, seed=SEED)

  model = Sequential()
  model.add(tf.keras.Input(shape=(input_dim,))) # Input tensor
  model.add(Dense(1, activation='sigmoid', kernel_initializer=initializer))  # Output tensor
  print(model.summary())

  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# Hpyer parameter Tuning
def create_pipeline_LinRegr(c_epochs, c_batch_size, m_input_dim):
  # Create KerasClassifier
  kerasclf_model = KerasClassifier(model=create_model_LinRegr, epochs=c_epochs,
                                   batch_size=c_batch_size, verbose=0,
                                   input_dim=m_input_dim)
  # Create the scikit-learn pipeline
  kpipe_lr = Pipeline(steps=[
      ('prep', ct),
       ('clf', kerasclf_model)
       ])
  return kpipe_lr

nepochs=[20, 50]
nbatchsizes=[8, 64]
hplist_lr = []
kpipes_lr = []
for n_ep in nepochs:
  for n_bsize in nbatchsizes:
    kpipe = create_pipeline_LinRegr(n_ep, n_bsize, num_pipe_feat)
    hplist_lr.append([n_ep, n_bsize])
    kpipes_lr.append(kpipe)
num_sets_lr = len(hplist_lr)

# Fit the pipeline on the training data
kc_scores_lr = []
kc_exectimes_lr = []
for i in range(num_sets_lr):
  print(f"Hyperparam Set{i}:  {hplist_lr[i]}")
  start_time = time.time()
  # fit data to pipeline model
  history = kpipes_lr[i].fit(X_train, y_train)
  # time to train
  kc_exectimes_lr.append(time.time() - start_time)
  # predict test data
  y_test_pipe_pred = kpipes_lr[i].predict(X_test)
  # get roc_auc_score
  kc_scores_lr.append(roc_auc_score(y_test, y_test_pipe_pred))
  print(f"LinRegr Index: {i}, {hplist_lr[i]}, score={kc_scores_lr[i]} exectime={kc_exectimes_lr[i]}")

Hyperparam Set0:  [20, 8]


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_44 (Dense)                     │ (None, 1)                   │             105 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 105 (420.00 B)

 Trainable params: 105 (420.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_47 (Dense)                     │ (None, 1)                   │             105 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 105 (420.00 B)

 Trainable params: 105 (420.00 B)

 Non-trainable params: 0 (0.00 B)

None
LinRegr Index: 3, [50, 64], score=0.7659585674915271 exectime=34.79126691818237


In [19]:
print("Scores with Linear Regression")
for i in range(num_sets_lr):
  print(f"Index: {i:2d}, {hplist_lr[i]}  score={kc_scores_lr[i]:0.4f}  exectime={kc_exectimes_lr[i]:4.2f}")

Scores with Linear Regression
Index:  0, [20, 8]  score=0.7778  exectime=93.57
Index:  1, [20, 64]  score=0.7543  exectime=14.45
Index:  2, [50, 8]  score=0.7785  exectime=232.58
Index:  3, [50, 64]  score=0.7660  exectime=34.79


In [20]:
!pip install xgboost
from xgboost import XGBClassifier

In [21]:
%env OBJC_DISABLE_INITIALIZE_FORK_SAFETY=YES
# Avoid - /usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38:
# RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded,
# so this will likely lead to a deadlock. pid = os.fork()

# XGBoost Classifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# Hpyer parameter Tuning
def create_pipeline_XGB():
  # Create model
  xgb_model = XGBClassifier()

  # Create the scikit-learn pipeline
  kpipe = Pipeline(steps=[
       ('prep', ct),
       ('clf', xgb_model)
       ])
  return kpipe

kpipe_xgb = create_pipeline_XGB()

xgb_params = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [1, 2, 4],
    'reg_lambda': [0, 1, 10],
    'tree_method': ["hist"],
    'enable_categorical': [True], 'random_state': [4]
    }
# Fit the pipeline on the training data
start_time = time.time()
# fit data to pipeline model
# history = kpipe_xgb.fit(X_train, y_train)
pipe_clf_params = {f'clf__{paramname}': paramvalue for paramname, paramvalue in xgb_params .items()}
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
# grid_search = GridSearchCV(kpipe_xgb, pipe_clf_params, scoring="accuracy", n_jobs=-1, cv=kfold)
grid_search = GridSearchCV(kpipe_xgb, pipe_clf_params, scoring="roc_auc",
                           cv=kfold, verbose=2)
grid_result = grid_search.fit(X_train, y_train)
# time to train
exectime_xgb = (time.time() - start_time)


env: OBJC_DISABLE_INITIALIZE_FORK_SAFETY=YES
Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END clf__enable_categorical=True, clf__learning_rate=0.1, clf__max_depth=1, clf__random_state=4, clf__reg_lambda=0, clf__tree_method=hist; total time=   0.7s
[CV] END clf__enable_categorical=True, clf__learning_rate=0.1, clf__max_depth=1, clf__random_state=4, clf__reg_lambda=0, clf__tree_method=hist; total time=   2.6s
[CV] END clf__enable_categorical=True, clf__learning_rate=0.1, clf__max_depth=1, clf__random_state=4, clf__reg_lambda=0, clf__tree_method=hist; total time=   4.8s
[CV] END clf__enable_categorical=True, clf__learning_rate=0.1, clf__max_depth=1, clf__random_state=4, clf__reg_lambda=0, clf__tree_method=hist; total time=   3.1s
[CV] END clf__enable_categorical=True, clf__learning_rate=0.1, clf__max_depth=1, clf__random_state=4, clf__reg_lambda=0, clf__tree_method=hist; total time=   2.0s
[CV] END clf__enable_categorical=True, clf__learning_rate=0.1, clf__max_depth=1

In [22]:
print("grid_result.best_params_", grid_result.best_params_)
print("grid_result.best_score_", grid_result.best_score_)

grid_result.best_params_ {'clf__enable_categorical': True, 'clf__learning_rate': 0.2, 'clf__max_depth': 4, 'clf__random_state': 4, 'clf__reg_lambda': 1, 'clf__tree_method': 'hist'}
grid_result.best_score_ 0.9262718189967151


In [39]:
# Hpyer parameter Tuning - use the best param from GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# Create model
xgb_model_best = XGBClassifier(enable_categorical=True, learning_rate=0.2,
                    max_depth=4, reg_lambda=1, tree_method='hist',
                    random_state=4)

# Create the scikit-learn pipeline
kpipe_xgb_best = Pipeline(steps=[
       ('prep', ct),
       ('clf', xgb_model_best)
       ])

start_time = time.time()
# fit data to pipeline model
history = kpipe_xgb_best.fit(X_train, y_train)
# time to train
exectime_xgb_best = (time.time() - start_time)

# predict test data
y_test_pipe_pred_xgb_best = kpipe_xgb_best.predict(X_test)

# get roc_auc_score
score_xgb_best = roc_auc_score(y_test, y_test_pipe_pred_xgb_best)
print(f"# XGBoost Classifier y_test_pipe_pred_xgb roc_auc_score={score_xgb_best} exectime={exectime_xgb_best} sec.")

# XGBoost Classifier y_test_pipe_pred_xgb roc_auc_score=0.8037797518741121 exectime=1.9994075298309326 sec.


In [34]:
# Hpyer parameter Tuning - use the best param from GridSearchCV - separate steps
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# Column Transformer fit transform on X
X_train_trans = ct.fit_transform(X_train)
X_test_trans = ct.transform(X_test)

# Create model
xgb_model_best = XGBClassifier(enable_categorical=True, learning_rate=0.2,
                    max_depth=4, reg_lambda=1, tree_method='hist',
                    random_state=4)

start_time = time.time()
# fit data to pipeline model
history = xgb_model_best.fit(X_train_trans, y_train)
# time to train
exectime_xgb_best = (time.time() - start_time)

# predict test data
y_test_pipe_pred_xgb_best = xgb_model_best.predict(X_test_trans)

# get roc_auc_score
score_xgb_best = roc_auc_score(y_test, y_test_pipe_pred_xgb_best)
print(f"# XGBoost Classifier y_test_pipe_pred_xgb roc_auc_score={score_xgb_best} exectime={exectime_xgb_best} sec.")

# XGBoost Classifier y_test_pipe_pred_xgb roc_auc_score=0.8037797518741121 exectime=0.36411190032958984 sec.


## Summary:
This is a continuation of the mini project on Deep Learning with the Adult income dataset, focusing on hyper parameter tuning.

Have the following:
 *  ColumnTransformer with OneHotEncoder for categorical features and MinMaxScaler for continuous features.
 *  function create_model(input_dim, num_layers=2, num_neurons=8, act_function='relu'), that returns a model to use
 *  function create_pipeline(m_input_dim, m_nl, m_nn, m_af), that creates a pipeline to preprocess with the ColumnTransformer followed by create_model
 *  function kpipe_fit_pred_score(n_ep, n_bsize, num_pipe_feat, m_nl, m_nn, m_af), that uses the pipeline to create the model, fits training data to given epochs and batch_sizes, predicts on the test data and returns the roc_auc_score and time to do the fit

Case 1:
 ColumnTransform X
 create_model for a **basic model with num_layers=1, num_neurons=8, act_function='relu' </br>
 fit with epochs=10 and batch_size=64. </br>
 roc_auc_score with basic_model: 0.894** </br>

Case 2:
 Tried different combinations of the following with a pipeline (ColumnTransformer and KerasClassifier) </br>
 [n_epochs, n_batchsize, m_nlayers, m_nunits, m_activationfunctions] </br>
 With this, **best roc_auc_score is score=0.8000** </br>
    **epochs=100  batch_size=64   nlayers=4  nunits=8  actfunc=sigmoid    score=0.8000 ** </br>

Case 3:
 Tried **Linear Regression with keras** with just an Input and Output tensor and a couple of epochs and batch sizes </br>
 With this, **best roc_auc_score is score=0.7659**</br>
   **epochs=100  batch_size=64 **

Case 4:
 Tried **XGBClassifier in the pipeline**, with a parameter_grid and GridSearchCV </br>
 Picked up params from the best grid_result score of 0.92627 </br>
   enable_categorical=True, learning_rate=0.2, max_depth=4, reg_lambda=1, tree_method='hist', random_state=4 </br>
 Then, tried fit with X_train and predict for X_test. </br>
 This gave an **roc_auc_score=0.80378**

Case 5:
 Tried **XGBClassifier without the pipeline** with separate steps. </br>
 This gave an roc_auc_score of **roc_auc_score=0.80378**

Without the pipeline, where the ColumnTransform is on the entire X (not Xtrain and Xtest separately),
the basic model gives a much better roc_auc_score of 0.894.  </br>
With the rest the roc_auc_score on the test data is around 0.8 not far from the LinearRegression value of 0.7659.

**For this dataset, it seems that all the above models are not far from each other in performance**
